In [1]:
from api.lib.blackjack import Table, Dealer, Player, DealerDecisionModel
from api.lib.decision_models.FixedStrategy import FixedStrategy
from api.lib.decision_models.FixedWithCardCounting import FixedWithCardCounting
from api.lib.decision_models.QDecisionModel import QDecisionModel
import tensorflow as tf

from matplotlib import pyplot as plt

2023-08-07 23:42:22.633281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 23:42:22.939485: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-07 23:42:22.939537: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-07 23:42:23.793035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [7]:

dealer = Dealer()
table = Table(dealer, 12, top_up_rate = 0.2)

policies = {
    "dealer": Player(DealerDecisionModel()), 
    "fixed": Player(FixedStrategy()), 
    "fixed w/count": Player(FixedWithCardCounting()), 
    "Q": Player(QDecisionModel(tf.keras.models.load_model("model/first_good_model"))), 
    "Q w/count": Player(QDecisionModel(tf.keras.models.load_model("model/second_with_counting"), use_hi_lo_count=True)) 
}

for key in policies.keys(): 
    table.add_player(policies[key])

In [8]:
balances = []
for i in range(len(table.players)): 
    balances.append([])
    
for i in range(4000): 
    table.play_round()
    if ((i+1) % 500 == 0): 
        print('round ', i+1)
    for n in range(len(table.players)): 
        balances[n].append(table.players[n].balance)

KeyboardInterrupt: 

In [ ]:

for i in range(len(table.players)): 
    print(f"player {i}: {table.players[i].balance}")

In [ ]:
#for i in range(len(table.players)): 
#    plt.plot(balances[i], label=f"player {i}")
plt.plot(balances[0], label="dealer model")
plt.plot(balances[1], label="fixed")
plt.plot(balances[2], label="fixed w/count")
plt.plot(balances[3], label="QLearning")
plt.plot(balances[4], label="QLearning with count")

#i = 0
#for key in policies.keys(): 
#    plt.plot(balances[i], label=key)
#    i += 1
plt.legend()
plt.show()